<a href="https://colab.research.google.com/github/stanfordnmbl/opencap-processing/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jumping technique analysis

In this example, we will compare different jumping techniques measured using OpenCap.

We will first set up our environment, then download and process OpenCap data, and finally analyze some results.

# Install OpenSim using condacolab

The first step is to install [condacolab](https://github.com/conda-incubator/condacolab) in your notebook. Once is this complete, you can use conda to install the OpenSim Conda package into the enviroment. Note, this might take a few minutes to complete.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c opensim_admin opensim

# Import the opencap-processing source code

This example is part of the opencap-processing Github repo. Let's clone this repo here and install some Python packages.

In [ ]:
!git clone https://github.com/stanfordnmbl/opencap-processing.git
%cd /content/opencap-processing
!python3 -m pip install -r requirements.txt

If the block above runs successfully, you should see that it installed several Python packages with a message like:

```
Successfully installed backcall-0.2.0 cycler-0.11.0 decorator-5.1.1 evdev-1.6.0 fonttools-4.37.4 ipython-7.34.0 jedi-0.18.1 kiwisolver-1.4.4 maskpass-0.3.6 matplotlib-3.5.3 matplotlib-inline-0.1.6 packaging-21.3 pandas-1.3.5 parso-0.8.3 pexpect-4.8.0 pickleshare-0.7.5 pillow-9.2.0 prompt-toolkit-3.0.31 ptyprocess-0.7.0 pygments-2.13.0 pynput-1.7.6 pyparsing-3.0.9 python-dateutil-2.8.2 python-decouple-3.6 python-xlib-0.31 pytz-2022.4 scipy-1.7.3 traitlets-5.4.0 typing-extensions-4.3.0 wcwidth-0.2.5.
```

# Import packages

Now that the correct Python packages have been installed and the source code has been cloned to this notebook, you can import packages specific to this script.

Note, you will be prompted to login to OpenCap with your account. If you have not created an account, you can visit [opencap.ai](https://opencap.ai) and click "Start Recording" to create an account. This will enable you to authenticate and analyze your own data in the future.

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import matplotlib.pyplot as plt
import utilsKinematics
from utils import download_kinematics

You should see a `Login successful` message.

# Download data

First, enter the session ID of the OpenCap data collection containing the jumping data (3ef5cfad-cf8a-420b-af15-2d833a33cfb8). This will be used to download the data directly from OpenCap. Then, run the block of code to download the data.

In [ ]:

# Specify session id; see end of url in app.opencap.ai/session/<session_id>.
session_id = "3ef5cfad-cf8a-420b-af15-2d833a33cfb8"

# Specify trial names in a list; use None to process all trials in a session.
specific_trial_names = None 

# Specify where to download the data.
data_folder = os.path.join("./../Data", session_id)

# Download data.
trial_names, modelName = download_kinematics(session_id, folder=data_folder, trialNames=specific_trial_names)

# Process data

Now that the data has been downloaded, you can extract kinematic parameters using OpenSim. To do so, we use a class named `utilsKinematics.py`. Take a look at the `utilsKinematics.py` code in the github repository to see all the things you can do.

For this example, we will extract the center of mass values, speeds, and accelerations of the different trials.

In [ ]:
# Get center of mass kinematics.
kinematics, center_of_mass = {}, {}
center_of_mass['values'], center_of_mass['speeds'], center_of_mass['accelerations'] = {}, {}, {}
for trial_name in trial_names:
    # Create object from class kinematics.
    kinematics[trial_name] = utilsKinematics.kinematics(data_folder, trial_name, modelName=modelName, lowpass_cutoff_frequency_for_coordinate_values=10)
    # Get center of mass values, speeds, and accelerations.
    center_of_mass['values'][trial_name] = kinematics[trial_name].get_center_of_mass_values(lowpass_cutoff_frequency=10)
    center_of_mass['speeds'][trial_name] = kinematics[trial_name].get_center_of_mass_speeds(lowpass_cutoff_frequency=10)
    center_of_mass['accelerations'][trial_name] = kinematics[trial_name].get_center_of_mass_accelerations(lowpass_cutoff_frequency=10)

# Analyze data (part 1)

Let's now compare the center of mass values and speeds between our two trials. Waht do you see? Does it make sense?

In [ ]:
# Plot center of mass vertical values and speeds.
fig, axs = plt.subplots(2, 1, figsize=(6, 6), sharex=True)
for trial_name in trial_names:
    # Align signals based on peak velocity.
    idx_peak_velocity = np.argmax(center_of_mass['speeds'][trial_name]['y'])
    time = center_of_mass['speeds'][trial_name]['time']
    x = time - time[idx_peak_velocity]
    # Plot center of mass values.
    y_values = center_of_mass['values'][trial_name]['y']
    y = y_values-y_values[0]
    axs[0].plot(x, y, label=trial_name, linewidth=3)
    # Plot center of mass speeds.
    y_speeds = center_of_mass['speeds'][trial_name]['y']
    y = y_speeds-y_speeds[0]
    axs[1].plot(x, y, label=trial_name, linewidth=3)
    
# Figure setttings.
for ax in axs:
    # Add labels.    
    ax.legend(fontsize=14)
    # Remove top and right borders.
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    # Change font size.
    ax.tick_params(axis='both', which='major', labelsize=16)
    # Change size labels.
    ax.xaxis.label.set_size(16)
    ax.yaxis.label.set_size(16)
# Add labels.
axs[0].set_ylabel('CoM position (m)')
axs[1].set_ylabel('CoM velocity (m/s)')
axs[1].set_xlabel('Time (s)')
fig.align_ylabels(axs)

# Analyze data (part 2)

Finally, let's compare the vertical forces results from the center of mass accelerations (F=ma). What do you see? Does it make sense? 

In [ ]:
# Plot vertical forces from accelerations (F=ma).
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
gravity = 9.81
for trial_name in trial_names:
    # Align signals based on peak velocity.
    idx_peak_velocity = np.argmax(center_of_mass['speeds'][trial_name]['y'])
    time = center_of_mass['speeds'][trial_name]['time']
    x = time - time[idx_peak_velocity]
    # Plot vertical ground reaction force.
    y_accelerations = center_of_mass['accelerations'][trial_name]['y']
    y = (y_accelerations + gravity) / gravity # vGRF expressed in bodyweights: vGRF = m(a+g)/mg
    ax.plot(x, y, label=trial_name, linewidth=3)

# Figure setttings.
# Add labels. 
ax.set_ylabel('vGRF (bodyweight)')
ax.set_xlabel('Time (s)')   
ax.legend(fontsize=14)
# Remove top and right borders.
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
# Change font size.
ax.tick_params(axis='both', which='major', labelsize=16)
# Change size labels.
ax.xaxis.label.set_size(16)
ax.yaxis.label.set_size(16)

Version 1.0

Creators: Antoine Falisse, Scott Uhlrich.
Last Updated on July 14, 2023

You can also find a non-Colab version of this script [here](https://github.com/stanfordnmbl/opencap-processing/blob/main/Examples/example_COM_analysis.py)

This notebook is made available under the [Apache 2.0 license](https://www.apache.org/licenses/LICENSE-2.0).